## One-Hot enconding

Usually a machine learning algoritmh understands numerical data only. Other kinds of data, also called categorical, carries a limited number of values. For example, if one wants to store car brands, the values would be categorical (because the answers would be things like Honda, Toyota, Ford, None, etc.). So this values must be encoded before being passed to a machine learning models in Python. 

The most popular standard Approach for Categorical Data is the *One-hot encoding*. One hot encoding creates new (binary) columns, indicating the presence of each possible value from the original data. It works very well unless your categorical variable takes on a large number of values (i.e. you generally won't it for variables taking more than 15 different values. It'd be a poor choice in some cases with fewer values, though that varies.)[1]
 
In this notebook we're using the home prices using dataset from Iowa, USA.

(Inspired by a tutorial published at www.kaggle.com)

[1] https://www.kaggle.com/dansbecker/using-categorical-data-with-one-hot-encoding/notebook

In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
import numpy as np
import pandas as pd
    
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

## 1. Loading datasets

In [3]:
df_train = pd.read_csv('data/iowa_train.csv')
df_test = pd.read_csv('data/iowa_test.csv')

# Drop houses where the target value for prediction is not defined (!)
df_train.dropna(axis=0, subset=['SalePrice'], inplace=True)

print("Train dataset size={} Test dataset size={}".format(df_train.shape[0], df_test.shape[0]))

Train dataset size=1460 Test dataset size=1459


## 2. Cleaning data and selecting features

Dropping columns with missing values. Also removing **Id** and **SalesPrice** as these columns does not matter.

In [4]:
cols_with_missing = [col for col in df_train.columns 
                                 if df_train[col].isnull().any()]                                  

# Loading predictors and target (X/y) 
X_train = df_train.drop(['Id', 'SalePrice'] + cols_with_missing, axis=1)
y_train = df_train.SalePrice
X_test  = df_test.drop(['Id'] + cols_with_missing, axis=1)

Now just considering numeric columns and columns with low cardinality, which is the number of unique values in a column. We use it as our only way to select categorical columns here. This is convenient, though a little arbitrary.

In [5]:
low_cardinality_cols = [cname for cname in X_train.columns if 
                                X_train[cname].nunique() < 10 and
                                X_train[cname].dtype == "object"]
numeric_cols = [cname for cname in X_train.columns if 
                                X_train[cname].dtype in ['int64', 'float64']]
cols = low_cardinality_cols + numeric_cols

X_train = X_train[cols]
X_test  = X_test[cols]
print("Amount of columns used as predictors = {}".format(len(cols)))
X_train.dtypes.sample(10)

Amount of columns used as predictors = 57


LowQualFinSF     int64
SaleType        object
Utilities       object
BsmtFinSF2       int64
LotShape        object
OpenPorchSF      int64
BsmtHalfBath     int64
GarageCars       int64
HouseStyle      object
LotConfig       object
dtype: object

### Function to return MAE using cross_val_score (k-folding)

In [6]:
def get_mae(X, y):
    # multiple by -1 to make positive MAE score instead of neg value returned as sklearn convention
    return -1 * cross_val_score(RandomForestRegressor(50), 
                                X, y, 
                                scoring = 'neg_mean_absolute_error').mean()

## 3. Trying some features set configurations

In  order to see how the configurations perform, it's calculated the **Mean absolute error** of models built with these different sets of features. One-Hot Encoding may vary on a case-by-case basis. In the Models 2 and 3, it seems not to have any great benefit from using the one-hot encoded variables.

### Model 1 - Not considering categorical columns


In [7]:
X_train_new = X_train.select_dtypes(exclude=['object'])

print('MAE of SalePrice when Dropping Categoricals = {:,.0f}'.format(get_mae(X_train_new, y_train)))

MAE of SalePrice when Dropping Categoricals = 18,562


### Model 2 - Using One-Hot Encoding (get_dummies)

In [8]:
X_train_new = pd.get_dummies(X_train)

print('MAE of SalePrice when Dropping Categoricals = {:,.0f}'.format(get_mae(X_train_new, y_train)))

MAE of SalePrice when Dropping Categoricals = 18,306


### Model 3 - Considering data from test dataset also


In [9]:
X_train_new = pd.get_dummies(X_train)
X_test_new = pd.get_dummies(X_test)
a,b = X_train_new.align(X_test_new, join='left', axis=1)